In [41]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split,ShuffleSplit,cross_val_score,KFold
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import scatter_matrix
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import math

In [45]:



print ("Start time :",datetime.now().strftime("%H:%M:%S"))

C_val = 1000.0

def get_margin_width(coeffs,intercept):
    sq_addition = 0
    sq = 0
    for i in range(len(coeffs[0])) :
        sq = i * i
        sq_addition = sq_addition + sq
    res_sqrt = math.sqrt(sq_addition)
    
    margin_width = 2/res_sqrt
    
    return margin_width


def run_model_classification(next_X4_train,next_Y4_train,X_train,y_train,test_set_X,test_set_Y):
    
    model5 =svm.SVC(kernel='linear',C=C_val)
    print ("**********Start time *********** :",datetime.now().strftime("%H:%M:%S"))
    model5.fit(next_X4_train, next_Y4_train)
    print ("**********End time *********** :",datetime.now().strftime("%H:%M:%S"))

    print("Model params are : ", model5, "\n")

    y5_pred = model5.predict(X_train)
    print("orig base validation :: \n",classification_report(y_train, y5_pred))
    conf_mat5 = confusion_matrix(y_train, y5_pred)
    print("orig base validation : confusion-matrix : \n",conf_mat5)


    y5_pred = model5.predict(next_X4_train)
    print("base validation : \n",classification_report(next_Y4_train, y5_pred))
    conf_mat5 = confusion_matrix(next_Y4_train, y5_pred)
    print("base validation , confusion matrix :: \n",conf_mat5)

    support_5 = model5.support_vectors_
    indexes_5 = model5.support_
    print("support-vectors shape",support_5.shape)
    print("test-set shape:",test_set_X.shape)
    test_set_X = np.append(test_set_X,model5.support_vectors_,axis=0)
    test_set_Y = np.append(test_set_Y,next_Y4_train[indexes_5,],axis=0)
    print("test-set shape:",test_set_X.shape)
    print("X :: test-set : \n","length : ",len(test_set_X))#,test_set_X)
    print(" Y :: test-set : \n","length : ",len(test_set_Y))#,test_set_Y)
    next_X5_train = np.delete(next_X4_train, indexes_5, axis=0)
    next_Y5_train = np.delete(next_Y4_train,indexes_5,axis =0)
    print("New train-set-len",len(next_X5_train))
    print("New train-set shape",next_X5_train.shape)

    y5_pred = model5.predict(test_set_X)
    print("support-vector : \n",classification_report(test_set_Y, y5_pred))
    conf_mat5 = confusion_matrix(test_set_Y, y5_pred)
    print("support-vector :  confusion matrix :: \n",conf_mat5)
    #fig, ax = plt.subplots(figsize=(10,10))

    y5_pred = model5.predict(next_X5_train)
    print("support-vector : \n",classification_report(next_Y5_train , y5_pred))
    conf_mat5 = confusion_matrix(next_Y5_train , y5_pred)
    print("support-vector :  confusion matrix :: \n",conf_mat5)
    
    #hyperplane_margin = 2/(||w||)
    print("Model coefficients : ", model5.coef_, " intercept : ", model5.intercept_)
    print("Margin width of hyperplane : ", get_margin_width(model5.coef_,model5.intercept_))

    return next_X5_train, next_Y5_train, test_set_X, test_set_Y


print("*******************************************")
print("*******************************************")

train_data = pd.read_csv(
'/ws/sundashr/outOfBox/parasparam/pps_orig_kb_binary_class_3_features.csv',sep= ','
)

print(train_data.head)


print ("Train Dataset Lenght:: ", len(train_data))
print ("Train Dataset Shape:: ", train_data.shape)


train_set = train_data.values[:]
print("train-set, len",train_set, len(train_set))

X = train_set[:, [1,2,3]]
print ("dataset: \n",X)
Y = train_set[:,0]
print ("labels: \n",Y)


X_train = X
y_train = Y


#normalize the data
print ("Start Data norm time :",datetime.now().strftime("%H:%M:%S"))
scaling_train = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
#scaling_test = MinMaxScaler(feature_range=(-1,1)).fit(X_test)
X_train = scaling_train.transform(X_train)
#X_test = scaling_test.transform(X_test)
print("Scaled x-train\n",X_train)
#print("Scaled x-test\n",X_test)
print ("End Data norm time :",datetime.now().strftime("%H:%M:%S"))


#iteration 1
model = svm.SVC(kernel='linear',C=C_val)
print("Iter 1 : created linear model, started linear fit on given dataset")
print ("**********Start time *********** :",datetime.now().strftime("%H:%M:%S"))
model.fit(X_train, y_train)
print ("**********End time *********** :",datetime.now().strftime("%H:%M:%S"))
print("Iter 1 : created linear model")
print("Model params are : ", model, "\n")
y_pred = model.predict(X_train)
print("base dataset validation report : \n" ,classification_report(y_train, y_pred))
conf_mat = confusion_matrix(y_train, y_pred)
print("iter-1 :base-dataset validation: confusion-matrix : \n",conf_mat)
support = model.support_vectors_
indexes = model.support_
print("support-vec shape : ",support.shape)
test_set_X = np.copy(model.support_vectors_)
print("test_set_X shape : ",test_set_X.shape)
test_set_Y = np.copy(y_train[indexes,])
print("support len",len(support),"indexes-lin",len(indexes))
print("indexes : \n",indexes)
print("iter-1 X :: test-set : \n","length : ",len(test_set_X))#,test_set_X)
print("iter-1 Y :: test-set : \n","length : ",len(test_set_Y))#,test_set_Y)
print("indexes[0]:",indexes[0])
print("support[0] : \n",support[0])
print("train-date[index] : \n",X_train[indexes[0]])
print("new test-data[0] : \n",test_set_X[0])


y_pred = model.predict(test_set_X )
print("iter-1 : support-vector predict dataset validation report : \n" ,classification_report(test_set_Y, y_pred))
conf_mat = confusion_matrix(test_set_Y, y_pred)
print("iter-1 :support-vector predict dataset validation: confusion-matrix : \n",conf_mat)


next_X_train = np.delete(X_train, indexes, axis=0)
next_Y_train = np.delete(y_train,indexes,axis =0)
print("New train-set-len",len(next_X_train))
print("New train-set shape",next_X_train.shape)


y_pred = model.predict(next_X_train )
print("iter-1 : non support-vector predict dataset validation report : \n" ,classification_report(next_Y_train , y_pred))
conf_mat = confusion_matrix(next_Y_train , y_pred)
print("iter-1 : non support-vector predict dataset validation: confusion-matrix : \n",conf_mat)


print("Coeffiients : model : ", model.coef_[0])
print("Intercept : model : ", model.intercept_[0])
print("Margin : model : ", get_margin_width(model.coef_,model.intercept_))

old_X_train = next_X_train
old_Y_train = next_Y_train
old_test_set_X =  test_set_X
old_test_set_Y = test_set_Y



for iter in range(2,15):
    print("****************************************************************************")
    print("ITERATION ", iter, " START")

    new_X_train, new_Y_train, new_test_set_X, new_test_set_Y = run_model_classification(old_X_train,
            old_Y_train,X_train,y_train,old_test_set_X,old_test_set_Y)
                 
    old_X_train = new_X_train
    old_Y_train = new_Y_train
    old_test_set_X = new_test_set_X
    old_test_set_Y = new_test_set_Y
    
    print("ITERATION ", iter, " END")
    print("****************************************************************************")
    



Start time : 00:40:51
*******************************************
*******************************************
<bound method NDFrame.head of       Label    pps           bps      derived
0        AB     10  6.981603e+02    70.083969
1        AB     11  7.572725e+02    68.953623
2        AB     12  7.596323e+02    63.364903
3        AB     13  8.618618e+02    66.475610
4        AB     14  9.028471e+02    64.426752
..      ...    ...           ...          ...
341  Attack    764  1.061015e+06  1388.000000
342  Attack   2041  1.102041e+06   540.000000
343      AB    766  1.148880e+06  1500.000000
344  Attack   2404  1.730769e+06   720.000000
345      AB  41667  2.916667e+06    70.000000

[346 rows x 4 columns]>
Train Dataset Lenght::  346
Train Dataset Shape::  (346, 4)
train-set, len [['AB' 10 698.1603052999999 70.08396947]
 ['AB' 11 757.2724638 68.95362319]
 ['AB' 12 759.632312 63.36490251]
 ...
 ['AB' 766 1148880.0 1500.0]
 ['Attack' 2404 1730769.0 720.0]
 ['AB' 41667 2916667.0 70.0]] 3

base validation : 
               precision    recall  f1-score   support

          AB       1.00      1.00      1.00       169
      Attack       1.00      1.00      1.00        21

    accuracy                           1.00       190
   macro avg       1.00      1.00      1.00       190
weighted avg       1.00      1.00      1.00       190

base validation , confusion matrix :: 
 [[169   0]
 [  0  21]]
support-vectors shape (3, 3)
test-set shape: (156, 3)
test-set shape: (159, 3)
X :: test-set : 
 length :  159
 Y :: test-set : 
 length :  159
New train-set-len 187
New train-set shape (187, 3)
support-vector : 
               precision    recall  f1-score   support

          AB       0.77      0.72      0.75        79
      Attack       0.74      0.79      0.76        80

    accuracy                           0.75       159
   macro avg       0.76      0.75      0.75       159
weighted avg       0.76      0.75      0.75       159

support-vector :  confusion matrix :: 
 [[57 22]


 length :  168
New train-set-len 178
New train-set shape (178, 3)
support-vector : 
               precision    recall  f1-score   support

          AB       0.78      0.73      0.76        83
      Attack       0.76      0.80      0.78        85

    accuracy                           0.77       168
   macro avg       0.77      0.77      0.77       168
weighted avg       0.77      0.77      0.77       168

support-vector :  confusion matrix :: 
 [[61 22]
 [17 68]]
support-vector : 
               precision    recall  f1-score   support

          AB       1.00      1.00      1.00       164
      Attack       1.00      1.00      1.00        14

    accuracy                           1.00       178
   macro avg       1.00      1.00      1.00       178
weighted avg       1.00      1.00      1.00       178

support-vector :  confusion matrix :: 
 [[164   0]
 [  0  14]]
Model coefficients :  [[0.01037511 0.149937   2.25871769]]  intercept :  [1.01138679]
Margin width of hyperplane :  0.89

**********Start time *********** : 00:40:52
**********End time *********** : 00:40:52
Model params are :  SVC(C=1000.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False) 

orig base validation :: 
               precision    recall  f1-score   support

          AB       0.93      0.91      0.92       247
      Attack       0.78      0.83      0.80        99

    accuracy                           0.88       346
   macro avg       0.86      0.87      0.86       346
weighted avg       0.89      0.88      0.89       346

orig base validation : confusion-matrix : 
 [[224  23]
 [ 17  82]]
base validation : 
               precision    recall  f1-score   support

          AB       1.00      1.00      1.00       161
      Attack       1.00      1.00      1.00        11

    accuracy                    

In [ ]:
#print(data.head())
#print (type(X))

In [ ]:
#y_final = [y_pred,y1_pred,y2_pred,y3_pred]


In [ ]:
#y_final_data = np.transpose(y_final)
#print (y_final_data)

In [ ]:
#pd.DataFrame(y_final_data).to_csv("Entire_data_predicted_1.csv", header=['volumetric', 'Packet arrive time', 'Packet sized','entire data'])